In [48]:
#ライブラリインポート
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import datetime
import time
import re
import lxml
import matplotlib.pyplot as plt
import gensim

In [88]:
#Uta-Net先頭URL
base_url = 'https://www.uta-net.com'
#歌詞一覧ページURL
artist_url = '/artist/'
all_song_list = []

In [89]:
# 34700まであるっぽい 1m/20
for i in range(34700):
    artist_number = str(i) + "/"
    url = base_url + artist_url + artist_number
    #歌詞一覧ページの取得
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('td', class_='sp-w-100 pt-0 pt-lg-2')
        for link in links:
            a = base_url + (link.a.get('href'))
            all_song_list.append(a)
    except:
        print(i)
        continue

In [168]:
len(all_song_list)

325723

In [174]:
#取得したデータを格納するデータフレームを作成
songs_df = pd.DataFrame(columns=['song_name','lyrics','release_date','impression'])

In [187]:
for a in all_song_list[283276:]:
    #歌詞ページよりHTMLを取得
    response_a = requests.get(a)
    soup_a = BeautifulSoup(response_a.text, 'lxml')
    #曲名取得
    try:
        song_name = soup_a.find('h2').text
    except:
        song_name = None
    #歌詞取得
    try:
        song_lyric = soup_a.find('div', itemprop='lyrics').text.replace('\n','').replace("\u3000", "").replace("この歌詞をマイ歌ネットに登録\xa0>このアーティストをマイ歌ネットに登録\xa0>", "")
    except:
        song_lyric = None
    #発売日取得
    try:
        detail = soup_a.find('p', class_='detail').text
        match = re.search(r'\d{4}/\d{2}/\d{2}', detail)
        release_date = datetime.datetime.strptime(match.group(), '%Y/%m/%d').date()
    except:
        release_date = None
    #表示回数を取得
    try:
        p = r'この曲の表示回数：(.*)回'
        impression = re.search(p, detail).group(1)
    except:
        impression = None
    #取得したデータフレームに追加
    temp_df = pd.DataFrame([[song_name], [song_lyric], [release_date], [impression]], index=songs_df.columns).T
    songs_df = songs_df.append(temp_df, ignore_index=True)
    #1秒待機
    time.sleep(2)

In [94]:
print(match)
283275

None


In [173]:
all_song_list[]

'https://www.uta-net.com/song/290768/'

In [189]:
songs_df.to_excel("songs5.xlsx")

In [5]:
df1 = pd.read_excel("songs1.xlsx")
df2 = pd.read_excel("songs2.xlsx")
df3 = pd.read_excel("songs3.xlsx")
df4 = pd.read_excel("songs4.xlsx")
df5 = pd.read_excel("songs5.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'songs1.xlsx'

In [6]:
df_concat = pd.concat([df1, df2, df3, df4, df5])

NameError: name 'df1' is not defined

In [37]:
df_concat = pd.read_excel("usedata.xlsx",engine="openpyxl")
df_concat = df_concat.drop(columns="Unnamed: 0")

In [23]:
song_name = df_concat["song_name"]
lyrics = df_concat["lyrics"]
release_date = df_concat["release_date"]
impression = df_concat["impression"]

In [39]:
df_concat = df_concat.sort_values("release_date")

In [11]:
song_names    = []
lyricses      = []
release_dates = []
impressions   = []

In [14]:
df_concat = pd.read_excel("result.xlsx", engine="openpyxl")
df_concat = df_concat.drop(columns="Unnamed: 0")
df_concat = df_concat.sort_values("release_date")
# 1986年1月〜2023年8月
df_concat = df_concat[267:322393]

In [51]:
lyrics_list = [i for i in df_concat["lyrics"]]
release_date = [i for i in df_concat["release_date"]]
release_date_list = []
for i in release_date:
    release_date_list.append(str(i).replace("-", "")[:6])
songs_list_4sq = []
for i in range(38):
    a = []
    for j in range(4):
        a.append([])
    songs_list_4sq.append(a)

# 文字列が多すぎるため3次元配列として持てないので配列のインデックス番号で保持。
for i, j in enumerate(release_date_list):
    year = int(j[:4])
    month = int(j[4:])
    year_num = year - 1986
    month_num = (month-1)//3
    songs_list_4sq[year_num][month_num].append(i)

In [54]:
# モデル1データセット 1986〜2000年
model1_songs_index = songs_list_4sq[:14]
model1_songs_list = lyrics_list[:34296]
# モデル2データセット 2000〜2023年
model2_songs_index = songs_list_4sq[14:]
model2_songs_list = lyrics_list[34296:]
# モデル3データセット 1986〜2023年
model3_songs_index = songs_list_4sq
model3_songs_list = lyrics_list

In [55]:
from janome.tokenizer import Tokenizer
model1_text_file = 'model1_songs_word_lists.txt'
model2_text_file = 'model2_songs_word_lists.txt'
model3_text_file = 'model3_songs_word_lists.txt'

In [42]:
def data_processing(song_lyrics, text_file):
    t = Tokenizer()
    results = []
    for s in song_lyrics:
        tokens = t.tokenize(s)
        r = []
        for tok in tokens:
            if tok.base_form == '*':
                word = tok.surface
            else:
                word = tok.base_form
            ps = tok.part_of_speech
            hinshi = ps.split(',')[0]
            if hinshi in ['名詞', '形容詞', '動詞', '副詞']:
                r.append(word)
        rl = (' '.join(r)).strip()
        results.append(rl)
    #余計な文字コードの置き換え
    result = [i.replace('\u3000','') for i in results]
    with open(text_file, 'w', encoding='utf-8') as fp:
        fp.write("\n".join(result))

In [79]:
data_processing(model1_songs_list, model1_text_file)
data_processing(model2_songs_list, model2_text_file)
data_processing(model3_songs_list, model3_text_file)

In [57]:
for i, j in enumerate(songs_list_4sq):
    i = i + 1986
    for k, l in enumerate(j):
        k = k + 1
        text_file_name = "4sq_songs/" + str(i) + str(k) + "_songs_word_lists.txt"
        t = Tokenizer()
        results = []
        for s in l:
            tokens = t.tokenize(lyrics_list[s])
            r = []
            for tok in tokens:
                if tok.base_form == '*':
                    word = tok.surface
                else:
                    word = tok.base_form
                ps = tok.part_of_speech
                hinshi = ps.split(',')[0]
                if hinshi in ['名詞', '形容詞', '動詞', '副詞']:
                    r.append(word)
            rl = (' '.join(r)).strip()
            results.append(rl)
        #余計な文字コードの置き換え
        result = [p.replace('\u3000','') for p in results]
        with open(text_file_name, 'w', encoding='utf-8') as fp:
            fp.write("\n".join(result))

In [1]:
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
from sklearn.cluster import KMeans

In [4]:
from gensim.models import word2vec
model1_save_bin = 'model1_save.bin'
model2_save_bin = 'model2_save.bin'
model3_save_bin = 'model3_save.bin'

In [4]:
def model_learning(text_file, save_bin):
    sentences = word2vec.Text8Corpus(text_file)
    model = word2vec.Word2Vec(sentences, size=200, window=10, hs=0, min_count=1, sg=1)
    model.save(save_bin)

In [8]:
model_learning(model1_text_file, model1_save_bin)
model_learning(model2_text_file, model2_save_bin)
model_learning(model3_text_file, model3_save_bin)

In [45]:
# 1986年から2020年までを学習として2020年から2023年までをテストとする（乖離度の確認）
def analyze(save_bin:str):
    model = gensim.models.Word2Vec.load(save_bin)
    commit = []
    for i in range(34):
        i = i + 1986
        for j in range(4):
            result = []
            j = j + 1
            text_name = '4sq_songs/' + str(i) + str(j) + '_songs_word_lists.txt'
            f = open(text_name, 'r', encoding='UTF-8')
            data = f.read()
            for d in data:
                try:
                    r = model.wv.get_vector(d)
                except:
                    try:
                        p = model.wv.most_similar(positive=[d][0][0])
                        r = model.wv.get_vector(p)
                    except:
                        continue
                result.append(r)
            ans = []
            for k in range(200):
                sum_k = 0
                for l in result:
                    sum_k = sum_k + l[k]
                sum_k = sum_k/len(result)
                ans.append(sum_k)
            commit.append(ans)
    return commit

In [46]:
# 2023年は第二四半期まで
def analyze_test(save_bin:str):
    model = gensim.models.Word2Vec.load(save_bin)
    commit = []
    for i in range(4):
        i = i + 2020
        for j in range(4):
            result = []
            j = j + 1
            if i == 2023 and j == 3:
                return commit
            text_name = '4sq_songs/' + str(i) + str(j) + '_songs_word_lists.txt'
            f = open(text_name, 'r', encoding='UTF-8')
            data = f.read()
            for d in data:
                try:
                    r = model.wv.get_vector(d)
                except:
                    try:
                        p = model.wv.most_similar(positive=[d][0][0])
                        r = model.wv.get_vector(p)
                    except:
                        continue
                result.append(r)
            ans = []
            for k in range(200):
                sum_k = 0
                for l in result:
                    sum_k = sum_k + l[k]
                sum_k = sum_k/len(result)
                ans.append(sum_k)
            commit.append(ans)

In [49]:
model1_explain = analyze(model1_save_bin)
model2_explain = analyze(model2_save_bin)
model3_explain = analyze(model3_save_bin)

In [109]:
len(model1_explain)

136

In [50]:
model1_explain_test = analyze_test(model1_save_bin)
model2_explain_test = analyze_test(model2_save_bin)
model3_explain_test = analyze_test(model3_save_bin)

In [98]:
len(model2_explain_test)

14

In [99]:
# 日本CPI
df_cpi = pd.read_excel("purpose_data/cpi.xlsx", engine="openpyxl", index_col=0)
# 年月日
cpi_year = df_cpi["Unnamed: 8"][13:655]
# 値
cpi_value = df_cpi.iloc[13:655,12]

In [100]:
# 1986年第1四半期〜2023年第2四半期
cpi_4sq_value = []
for i, j in enumerate(cpi_year[192:]):
    if i%3 == 0:
        a = cpi_value[192+i]
    elif i%3 == 1:
        a += cpi_value[192+i]
    else:
        a += cpi_value[192+i]
        a = a/3
        cpi_4sq_value.append(a)

In [105]:
# 1986~2020
cpi_1986_2020 = cpi_4sq_value[:136]
cpi_2020_2023 = cpi_4sq_value[136:]

In [37]:
# 日本経済指標(CI値)
# 1985年から月ベースでの調査
df_japan_index = pd.read_excel("purpose_data/economy_index.xlsx", engine="openpyxl", index_col=0)
# 年
japan_index_year = df_japan_index["Unnamed: 1"][77:527]
# 月
japan_index_month = df_japan_index["Unnamed: 2"][77:527]
# 値
japan_index_value = df_japan_index["ＣＩ指数"][77:527].tolist()

In [39]:
# 1986年第1四半期〜2023年第2四半期
japan_index_4sq_value = []
for i, j in enumerate(japan_index_value):
    if i%3 == 0:
        a = j
    elif i%3 == 1:
        a += j
    else:
        a += j
        a = a/3
        japan_index_4sq_value.append(a)

In [43]:
ci_1986_2020 = japan_index_4sq_value[:136]
ci_2020_2023 = japan_index_4sq_value[136:]

In [162]:
# 日本GDP
df_gdp = pd.read_excel("purpose_data/GDP_4sq.xlsx", engine="openpyxl")
# 年四半期
gdp_4sq = df_gdp["name1"][7:124]
# 値
gdp_4sq_value = df_gdp["name2"][7:124]
gdp_list = []
for p in gdp_4sq_value:
    gdp_list.append(p)

In [166]:
gdp_4sq_1994_2020 = gdp_list[:104]
gdp_4sq_2020_2023 = gdp_list[104:]

In [174]:
# 1994年第1四半期〜2023年第1四半期
len(gdp_4sq_1994_2020)

104

In [3]:
# 日経平均株価
df_japan_ave = pd.read_excel("purpose_data/japan_avarage.xlsx", engine="openpyxl")
# 年月日
japan_ave_year = df_japan_ave["データ日付"][:282]
# 終わり値
japan_ave_index = df_japan_ave["終値"][:282]

In [161]:
# 2000年第1四半期〜2023年第2四半期
japan_ave_index_4sq = []
for i, j in enumerate(japan_ave_year):
    if i%3 == 0:
        a = japan_ave_index[i]
    elif i%3 == 1:
        a += japan_ave_index[i]
    else:
        a += japan_ave_index[i]
        a = a/3
        japan_ave_index_4sq.append(a)

## 重回帰分析

In [73]:
value_list = model1_explain

In [74]:
len(value_list)

136

In [75]:
import numpy as np
value_list_T = np.array(value_list).T

In [76]:
# 正規化
# 標準化
# 最大最小はすでに決まっているので標準化のみ
std_value_list_T = []
for i in value_list_T:
    a = (i-i.mean())/i.std(ddof=0)
    std_value_list_T.append(a)

In [77]:
y = ci_1986_2020

In [78]:
from sklearn.linear_model import LinearRegression

In [79]:
model_lr_std = LinearRegression()
x = np.array(std_value_list_T).T
model_lr_std.fit(x, y)
print(model_lr_std.coef_)
print(model_lr_std.intercept_) 
print(model_lr_std.score(x, y))

[-26.52764202  43.5545173    6.63122866 -22.61185456  13.65888712
  18.86550851  16.64970995 -22.75474753   2.42538278  -1.6100143
   2.8235949    2.85039971 -23.30532079  -0.13130902  -8.34734382
   9.10295301 -37.83399867 -35.37664145 -16.08667775 -25.80636191
   5.00621541  10.43024133   6.27318884  -3.99809023 -30.38602843
  -2.36142878  -6.77212128   1.78271698  -5.01498671  14.44072932
 -25.58541229 -24.53457258  -5.0231887  -10.09488363  11.75658573
 -25.64476321  -3.71077838 -20.08885407 -37.50848378 -33.9325374
   7.6020846   -4.77357324 -10.76532947 -17.73300018   5.98485892
   5.5650849    3.95950325  -0.22383942   8.15570972  11.52945409
 -17.87105149 -21.20446811  -1.73978292   5.31146416  -3.62320451
 -38.47455234  33.45539148 -13.65341936   3.96955264  20.24611417
  10.47991932 -31.53579296   3.18549417 -13.213505    -0.36340479
  -3.62958329  38.23245301   5.73839709   7.62330918  -5.20557774
  -0.58192562   9.46256034   5.54883206 -55.02554083  -7.75871106
  -6.0683956

In [164]:
def test():
    a = 1
    model3_explain_test

In [80]:
b = []
for i in model1_explain_test[:-1]:
    a = 0
    for p in range(200):
        a += i[p]*model_lr_std.coef_[p]
    a += model_lr_std.intercept_
    b.append(a)

In [81]:
len(b)

13

In [82]:
c = []
for i,j in enumerate(b):
    c.append(ci_2020_2023[i]-j)

In [83]:
c

[14.12429553121099,
 2.8934557079212766,
 14.617509852621552,
 21.053362804315015,
 27.304249256043192,
 28.153048217638613,
 28.087002343146096,
 28.01161911850231,
 26.67285318051391,
 26.380182670646747,
 24.339284319553343,
 24.49631755037312,
 21.919743442622064]

In [ ]:
102.75808823529364
102.75808823529377
102.75808823529378